In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
#!pip install tensorflow_addons

In [0]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.layers import Conv1D, Input, Dense, Add, Multiply, concatenate, Reshape
from tensorflow.keras.layers import Embedding, Bidirectional
from tensorflow.compat.v1.keras.layers import CuDNNGRU
from tensorflow.keras.callbacks import Callback, LearningRateScheduler
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K
from tensorflow.keras import losses, models, optimizers
import tensorflow_addons as tfa

In [0]:
# =====================================================================================
# Directory Settings
# =====================================================================================
DRIVE_PATH = './drive/My Drive/Colab Notebooks/Ion/'
ROOT = DRIVE_PATH+'input/liverpool-ion-switching/'
CLEAN_ROOT = DRIVE_PATH+'input/data-without-drift/'
OUTPUT_DIR = DRIVE_PATH+'stack1/'

In [0]:
class CFG:
    learning_rate=1e-3
    batch_size=16
    num_workers=4
    num_train_epochs=110
    weight_decay=0.01
    dropout=0.3
    emb_size=100
    hidden_size=164
    seq_len=5000
    total_cate_size=40
    seed=1225
    encoder='Wavenet_keras'
    target_size=11
    n_fold=4
    fold=[0, 1, 2, 3]

In [0]:
# =====================================================================================
# Library
# =====================================================================================
import sys
import os
import gc
import time
from contextlib import contextmanager
from collections import Counter, defaultdict

import numpy as np
import pandas as pd
import random
import json

from sklearn.model_selection import StratifiedKFold, GroupKFold
from sklearn import preprocessing

from tqdm import tqdm, tqdm_notebook

import torch

import warnings
warnings.filterwarnings("ignore")

# =====================================================================================
# Utils
# =====================================================================================
def get_logger(filename=OUTPUT_DIR+'log'):
    from logging import getLogger, INFO, StreamHandler, FileHandler, Formatter
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=f"{filename}.log")
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger

logger = get_logger()


@contextmanager
def timer(name):
    t0 = time.time()
    logger.info(f'[{name}] start')
    yield
    logger.info(f'[{name}] done in {time.time() - t0:.0f} s')


# for tf
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    tf.random.set_seed(seed)


def load_df(path, debug=False):
    # load df .csv or .pkl
    if path.split('.')[-1]=='csv':
        df = pd.read_csv(path)
        if debug:
            df = pd.read_csv(path, nrows=1000)
    elif path.split('.')[-1]=='pkl':
        df = pd.read_pickle(path)
    print(f"{path} shape / {df.shape} ")
    return df


def make_stratified_group_k_folds(_df, _id, target, group, k, seed=42, save_path=OUTPUT_DIR+'folds.csv'):

    def stratified_group_k_fold(X, y, groups, k, seed=42):

        """
        original author : jakubwasikowski
        reference : https://www.kaggle.com/jakubwasikowski/stratified-group-k-fold-cross-validation
        """

        labels_num = np.max(y) + 1
        y_counts_per_group = defaultdict(lambda: np.zeros(labels_num))
        y_distr = Counter()
        for label, g in zip(y, groups):
            y_counts_per_group[g][label] += 1
            y_distr[label] += 1

        y_counts_per_fold = defaultdict(lambda: np.zeros(labels_num))
        groups_per_fold = defaultdict(set)

        def eval_y_counts_per_fold(y_counts, fold):
            y_counts_per_fold[fold] += y_counts
            std_per_label = []
            for label in range(labels_num):
                label_std = np.std([y_counts_per_fold[i][label] / y_distr[label] for i in range(k)])
                std_per_label.append(label_std)
            y_counts_per_fold[fold] -= y_counts
            return np.mean(std_per_label)

        groups_and_y_counts = list(y_counts_per_group.items())
        random.Random(seed).shuffle(groups_and_y_counts)

        for g, y_counts in sorted(groups_and_y_counts, key=lambda x: -np.std(x[1])):
            best_fold = None
            min_eval = None
            for i in range(k):
                fold_eval = eval_y_counts_per_fold(y_counts, i)
                if min_eval is None or fold_eval < min_eval:
                    min_eval = fold_eval
                    best_fold = i
            y_counts_per_fold[best_fold] += y_counts
            groups_per_fold[best_fold].add(g)

        all_groups = set(groups)
        for i in range(k):
            train_groups = all_groups - groups_per_fold[i]
            test_groups = groups_per_fold[i]

            train_indices = [i for i, g in enumerate(groups) if g in train_groups]
            test_indices = [i for i, g in enumerate(groups) if g in test_groups]

            yield train_indices, test_indices

    df = _df.copy()
    le = preprocessing.LabelEncoder()
    groups = le.fit_transform(df[group].values)
    for n, (train_index, val_index) in enumerate(stratified_group_k_fold(df, df[target], groups, k=k, seed=seed)):
        df.loc[val_index, 'fold'] = int(n)
    df['fold'] = df['fold'].astype(int)
    df[[_id, target, group, 'fold']].to_csv(save_path, index=None)

    return df[[_id, target, group, 'fold']]

In [0]:
# =====================================================================================
# General Settings
# =====================================================================================
df_path_dict = {'train': CLEAN_ROOT+'train_clean.csv',
                'test': CLEAN_ROOT+'test_clean.csv',
                'sample_submission': ROOT+'sample_submission.csv'}
ID = 'time'
TARGET = 'open_channels'
seed_everything(seed=CFG.seed)


# =====================================================================================
# Data Loading
# =====================================================================================
with timer('Data Loading'):
    X_train = pd.read_csv(DRIVE_PATH+'input/data-without-drift/train_clean.csv')
    oof_lgb = pd.read_csv(DRIVE_PATH+'input/ion-oof/oof_lightgbm.csv').rename(columns={TARGET:'signal_lgb_oof'})
    oof_cat = pd.read_csv(DRIVE_PATH+'input/ion-oof/oof_catboost.csv').rename(columns={TARGET:'signal_cat_oof'})
    
    X_train = pd.concat([X_train, oof_lgb['signal_lgb_oof']], axis=1)
    X_train = pd.concat([X_train, oof_cat['signal_cat_oof']], axis=1)

    del oof_lgb, oof_cat; gc.collect()

[Data Loading] start
[Data Loading] done in 4 s


In [0]:
# =====================================================================================
# Preprocess
# =====================================================================================
X_train['batch'] = X_train.index // 500000
X_train['batch'] = X_train['batch'].astype(int)
#X_test['batch'] = X_test.index // 500000
#X_test['batch'] = X_test['batch'].astype(int)


def signal2cate(X_train, X_test=None, NUM_BINS=1000):
    signal_bins = np.linspace(X_train['signal'].min(), X_train['signal'].max(), NUM_BINS + 1)
    train_signal_dig = np.digitize(X_train['signal'], bins=signal_bins) - 1
    train_signal_dig = np.minimum(train_signal_dig, len(signal_bins) - 2)
    X_train['signal_cate'] = train_signal_dig
    if X_test is not None:
        test_signal_dig = np.digitize(X_test['signal'], bins=signal_bins) - 1
        test_signal_dig = np.minimum(test_signal_dig, len(signal_bins) - 2)
        X_test['signal_cate'] = test_signal_dig
        return  X_train, X_test
    return X_train

X_train = signal2cate(X_train, X_test=None, NUM_BINS=CFG.total_cate_size)


def add_num_features(X_train, X_test=None):
    max_signal = X_train['signal'].max()
    min_signal = X_train['signal'].min()
    X_train['signal_diff_max'] = max_signal - X_train['signal']
    X_train['signal_diff_min'] = min_signal - X_train['signal']
    if X_test is not None:
        X_test['signal_diff_max'] = max_signal - X_test['signal']
        X_test['signal_diff_min'] = min_signal - X_test['signal']
        return  X_train, X_test
    return X_train

X_train = add_num_features(X_train, X_test=None)


def calc_gradients(df):

    df['signal_gradient'] = np.gradient(df['signal'].values)

    return df

def preprocess_df(df):

    output = pd.DataFrame()

    for i in range(int(len(df)/500000)):
        tmp = df.loc[i * 500000: 500000*(i + 1) - 1].reset_index(drop=True)
        tmp = calc_gradients(tmp)
        output = pd.concat([output, tmp])

    return output.reset_index(drop=True)

X_train = preprocess_df(X_train)
#X_test = preprocess_df(X_test)

In [0]:
# =====================================================================================
# Train functions
# =====================================================================================
import math
from sklearn.metrics import f1_score

def lr_schedule(epoch):
    if epoch < 30:
        lr = CFG.learning_rate
    elif epoch < 40:
        lr = CFG.learning_rate / 3
    elif epoch < 50:
        lr = CFG.learning_rate / 5
    elif epoch < 60:
        lr = CFG.learning_rate / 7
    elif epoch < 70:
        lr = CFG.learning_rate / 9
    elif epoch < 80:
        lr = CFG.learning_rate / 11
    elif epoch < 90:
        lr = CFG.learning_rate / 13
    else:
        lr = CFG.learning_rate / 100
    return lr

class MacroF1(Callback):
    def __init__(self, model, inputs, targets):
        self.model = model
        self.inputs = inputs
        self.targets = np.argmax(targets, axis=2).reshape(-1)
        
    def on_epoch_end(self, epoch, logs):
        pred = np.argmax(self.model.predict(self.inputs), axis=2).reshape(-1)
        score = f1_score(self.targets, pred, average='macro')
        print(f' - F1Score: {score:.5f}')

In [0]:
# =====================================================================================
# Wavenet Model
# =====================================================================================
def Wavenet_keras(cate_cols, cont_cols):
    
    def wave_block(x, filters, kernel_size, n):
        dilation_rates = [2**i for i in range(n)]
        x = Conv1D(filters = filters,
                   kernel_size = 1,
                   padding = 'same')(x)
        res_x = x
        for dilation_rate in dilation_rates:
            tanh_out = Conv1D(filters = filters,
                              kernel_size = kernel_size,
                              padding = 'same', 
                              activation = 'tanh', 
                              dilation_rate = dilation_rate)(x)
            sigm_out = Conv1D(filters = filters,
                              kernel_size = kernel_size,
                              padding = 'same',
                              activation = 'sigmoid', 
                              dilation_rate = dilation_rate)(x)
            x = Multiply()([tanh_out, sigm_out])
            x = Conv1D(filters = filters,
                       kernel_size = 1,
                       padding = 'same')(x)
            res_x = Add()([res_x, x])
        return res_x
    
    input1 = Input(shape=(None, len(cont_cols)))
    input2 = Input(shape=(None, len(cate_cols)))
    # RNN
    cate_emb = Embedding(CFG.total_cate_size, CFG.emb_size)(input2)
    cate_emb = Reshape((-1, CFG.emb_size*(len(cate_cols))))(cate_emb)
    seq_emb = concatenate([cate_emb, input1])
    h_gru = Bidirectional(CuDNNGRU(CFG.hidden_size//4, return_sequences=True))(seq_emb) # dropout?
    # CNN
    cont_x = wave_block(input1, CFG.hidden_size//16, 3, 12)
    cont_x = wave_block(cont_x, CFG.hidden_size//8, 3, 8)
    cont_x = wave_block(cont_x, CFG.hidden_size//4, 3, 4)
    cont_x = wave_block(cont_x, CFG.hidden_size//2, 3, 1)
    # CNN & RNN
    x = concatenate([cont_x, h_gru])
    # fc
    x = Dense(11, activation='softmax', name='out')(x)
    
    model = models.Model(inputs=[input1, input2], outputs=x)
    
    opt = Adam(lr=CFG.learning_rate)
    opt = tfa.optimizers.SWA(opt)
    model.compile(loss=losses.CategoricalCrossentropy(), optimizer=opt, metrics=['accuracy'])

    return model

In [0]:
# =====================================================================================
# Get Sample function
# =====================================================================================
def tta_group(X_train):
    tmp = X_train[CFG.seq_len//2:len(X_train)-CFG.seq_len//2].reset_index(drop=True)
    tmp['tta_group'] = tmp.index // CFG.seq_len
    tmp['tta_group'] = tmp['tta_group'] + tmp['group'].nunique()
    X_train = pd.concat([X_train[:CFG.seq_len//2], 
                         tmp, 
                         X_train[len(X_train)-CFG.seq_len//2:]]).reset_index(drop=True).fillna(-1)
    return X_train

"""
def tta_group(X_train):
    X_train = X_train.reset_index()
    tmp = pd.DataFrame()
    for i in tqdm(range(int(len(X_train)/500000))):
        _tmp = X_train[i*500000+CFG.seq_len//2:(i+1)*500000-CFG.seq_len//2].reset_index(drop=True)
        tmp = pd.concat([tmp, _tmp])
    tmp = tmp.reset_index(drop=True)
    tmp['tta_group'] = tmp.index // CFG.seq_len
    tmp['tta_group'] = tmp['tta_group'] + X_train['group'].nunique()
    X_train = X_train.merge(tmp[['index', 'tta_group']], on='index', how='left').fillna(-1).drop(columns='index')
    return X_train
"""

def get_sample_indices(df):
    sample_indices = []
    group_indices = []
    df_groups = df.groupby('group').groups
    tta_df_groups = df[df['tta_group']>=0].groupby('tta_group').groups
    for group_idx, indices in enumerate(df_groups.values()):
        sample_indices.append(indices.values)
        group_indices.append(group_idx)
    for group_idx, indices in enumerate(tta_df_groups.values()):
        sample_indices.append(indices.values)
        group_indices.append(group_idx+len(df_groups))
    return np.array(sample_indices), group_indices

In [0]:
# =====================================================================================
# Train loop
# =====================================================================================
def main(X_train, folds):

    seed_everything(CFG.seed)
    K.clear_session()
    config = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1,inter_op_parallelism_threads=1)
    sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=config)
    tf.compat.v1.keras.backend.set_session(sess)

    # =====================================================================================
    # Settings
    # =====================================================================================
    cate_cols = ['signal_cate']
    cont_cols = [c for c in X_train.columns if c.find('signal')>=0]
    cont_cols = [c for c in cont_cols if c not in cate_cols]
    logger.info(f'cont_cols: {cont_cols}')
    logger.info(f'cate_cols: {cate_cols}')
    target_df = pd.get_dummies(X_train['open_channels'].values)
    cont_df = X_train[cont_cols]
    cate_df = X_train[cate_cols]

    # =====================================================================================
    # run function
    # =====================================================================================
    def run(fold, trn_idx, val_idx):

        train_cont_x = np.zeros((len(trn_idx), CFG.seq_len, len(cont_cols)))
        train_cate_x = np.zeros((len(trn_idx), CFG.seq_len, len(cate_cols)))
        train_y = np.zeros((len(trn_idx), CFG.seq_len, 11))
        for i in tqdm_notebook(range(len(trn_idx))):
            train_cont_x[i] = cont_df.iloc[sample_indices[trn_idx[i]]].values
            train_cate_x[i] = cate_df.iloc[sample_indices[trn_idx[i]]].values     
            train_y[i] = target_df.iloc[sample_indices[trn_idx[i]]].values
        
        val_cont_x = np.zeros((len(val_idx), CFG.seq_len, len(cont_cols)))
        val_cate_x = np.zeros((len(val_idx), CFG.seq_len, len(cate_cols)))
        val_y = np.zeros((len(val_idx), CFG.seq_len, 11))
        for i in tqdm_notebook(range(len(val_idx))):
            val_cont_x[i] = cont_df.iloc[sample_indices[val_idx[i]]].values
            val_cate_x[i] = cate_df.iloc[sample_indices[val_idx[i]]].values     
            val_y[i] = target_df.iloc[sample_indices[val_idx[i]]].values

        gc.collect()

        # =====================================================================================
        # Training loop
        # =====================================================================================
        model = Wavenet_keras(cate_cols, cont_cols)
        cb_lr_schedule = LearningRateScheduler(lr_schedule)
        loss_saving_callback = keras.callbacks.ModelCheckpoint(OUTPUT_DIR+f'keras_fold{fold}_best_loss.h5', monitor='val_loss', verbose=1, 
                                                               save_best_only=True, save_weights_only=True)
        score_saving_callback = keras.callbacks.ModelCheckpoint(OUTPUT_DIR+f'keras_fold{fold}_best_score.h5', monitor='val_accuracy', verbose=1, 
                                                                save_best_only=True, save_weights_only=True)
        # ModelCheckpoint追加するとMacroF1Callbackが出力されない...
        model.fit([train_cont_x, train_cate_x], train_y, epochs=CFG.num_train_epochs,
                  #callbacks=[cb_lr_schedule, loss_saving_callback, score_saving_callback, MacroF1(model, [val_cont_x, val_cate_x], val_y)],
                  callbacks=[cb_lr_schedule, loss_saving_callback, score_saving_callback],
                  batch_size=CFG.batch_size, verbose=1, validation_data=([val_cont_x, val_cate_x], val_y))
        #model.save_weights(OUTPUT_DIR+f'keras_fold{fold}.h5')

        # =====================================================================================
        # Evaluation
        # =====================================================================================
        model = Wavenet_keras(cate_cols, cont_cols)
        model.load_weights(OUTPUT_DIR+f'keras_fold{fold}_best_score.h5')
        preds_f = model.predict([val_cont_x, val_cate_x])
        predictions = np.argmax(preds_f, axis=2).reshape(-1)
        groundtruth = np.argmax(val_y, axis=2).reshape(-1)
        score = f1_score(predictions, groundtruth, labels=list(range(11)), average='macro')
        logger.info(f'Training fold {fold} completed. macro f1 score : {score :1.5f}')

        return predictions, groundtruth

    # =====================================================================================
    # k-fold
    # =====================================================================================
    folds = tta_group(folds)
    folds['tta_group'] = folds['tta_group'].astype(int)
    sample_indices, group_indices = get_sample_indices(folds)
    print(len(group_indices))
    print(len(set(group_indices)))
    group_map = dict(folds[['group', 'split_group']].values.tolist())
    tta_group_map = dict(folds[['tta_group', 'split_group']].values.tolist())
    group_map.update(tta_group_map)
    group_indices = [group_map[i] for i in group_indices]
    print(len(group_indices))
    print(len(set(group_indices)))
    folds_map = dict(folds[['split_group', 'fold']].values.tolist())
    group_indices = [folds_map[i] for i in group_indices]
    skf = GroupKFold(n_splits=CFG.n_fold)
    splits = [x for x in skf.split(sample_indices, None, group_indices)]
    predictions, groundtruth = [], []
    for fold, (trn_idx, val_idx) in enumerate(splits):
        if fold in CFG.fold:
            # with normal val_idx
            val_idx = val_idx[val_idx<(5000000//CFG.seq_len)]
            with timer(f'##### Running Fold: {fold} #####'):
                _predictions, _groundtruth = run(fold, trn_idx, val_idx)
                predictions.append(_predictions)
                groundtruth.append(_groundtruth)
    predictions = np.concatenate(predictions)
    groundtruth = np.concatenate(groundtruth)
    score = f1_score(predictions, groundtruth, labels=list(range(11)), average='macro')
    logger.info(f'##### CV Score: {score} #####')

In [13]:
if __name__ == '__main__':
    folds = pd.read_csv(DRIVE_PATH+'input/ion-folds/folds.csv')
    folds['group'] = folds.index // CFG.seq_len
    main(X_train, folds)

cont_cols: ['signal', 'signal_lgb_oof', 'signal_cat_oof', 'signal_diff_max', 'signal_diff_min', 'signal_gradient']
cate_cols: ['signal_cate']


1999
1999
1999
500


[##### Running Fold: 0 #####] start



Epoch 1/110
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
94/94 [==============================] - ETA: 0s - loss: 0.7072 - accuracy: 0.7811
Epoch 00001: val_loss improved from inf to 0.16461, saving model to ./drive/My Drive/Colab Notebooks/Ion/stack1/keras_fold0_best_loss.h5

Epoch 00001: val_accuracy improved from -inf to 0.95279, saving model to ./drive/My Drive/Colab Notebooks/Ion/stack1/keras_fold0_best_score.h5
94/94 [==============================] - 61s 645ms/step - loss: 0.7072 - accuracy: 0.7811 - val_loss: 0.1646 - val_accuracy: 0.9528 - lr: 0.0010
Epoch 2/110
94/94 [==============================] - ETA: 0s - loss: 0.1150 - accuracy: 0.9636
Epoch 00002: val_loss improved from 0.16461 to 0.10699, saving model to ./drive/My Drive/Colab Notebooks/Ion/stack1/keras_fold0_best_loss.h5

Epoch 00002: val_accuracy improved from 0.95279 to 0.96339, saving model to ./drive/My Drive/Colab Notebooks/Ion/stack1/keras_fold0_best_score.h5
94/94 [=======

Training fold 0 completed. macro f1 score : 0.93960
[##### Running Fold: 0 #####] done in 5624 s
[##### Running Fold: 1 #####] start


...

Training fold 1 completed. macro f1 score : 0.94262
[##### Running Fold: 1 #####] done in 5623 s
[##### Running Fold: 2 #####] start



Epoch 1/110
94/94 [==============================] - ETA: 0s - loss: 0.6643 - accuracy: 0.7858
Epoch 00001: val_loss improved from inf to 0.13766, saving model to ./drive/My Drive/Colab Notebooks/Ion/stack1/keras_fold2_best_loss.h5

Epoch 00001: val_accuracy improved from -inf to 0.96206, saving model to ./drive/My Drive/Colab Notebooks/Ion/stack1/keras_fold2_best_score.h5
94/94 [==============================] - 53s 566ms/step - loss: 0.6643 - accuracy: 0.7858 - val_loss: 0.1377 - val_accuracy: 0.9621 - lr: 0.0010
Epoch 2/110
94/94 [==============================] - ETA: 0s - loss: 0.1074 - accuracy: 0.9654
Epoch 00002: val_loss improved from 0.13766 to 0.09426, saving model to ./drive/My Drive/Colab Notebooks/Ion/stack1/keras_fold2_best_loss.h5

Epoch 00002: val_accuracy improved from 0.96206 to 0.96676, saving model to ./drive/My Drive/Colab Notebooks/Ion/stack1/keras_fold2_best_score.h5
94/94 [==============================] - 51s 540ms/step - loss: 0.1074 - accuracy: 0.9654 - val

Training fold 2 completed. macro f1 score : 0.94162
[##### Running Fold: 2 #####] done in 5666 s
[##### Running Fold: 3 #####] start



Epoch 1/110
94/94 [==============================] - ETA: 0s - loss: 0.6047 - accuracy: 0.8104
Epoch 00001: val_loss improved from inf to 0.15157, saving model to ./drive/My Drive/Colab Notebooks/Ion/stack1/keras_fold3_best_loss.h5

Epoch 00001: val_accuracy improved from -inf to 0.95613, saving model to ./drive/My Drive/Colab Notebooks/Ion/stack1/keras_fold3_best_score.h5
94/94 [==============================] - 59s 625ms/step - loss: 0.6047 - accuracy: 0.8104 - val_loss: 0.1516 - val_accuracy: 0.9561 - lr: 0.0010
Epoch 2/110
94/94 [==============================] - ETA: 0s - loss: 0.1144 - accuracy: 0.9627
Epoch 00002: val_loss improved from 0.15157 to 0.09936, saving model to ./drive/My Drive/Colab Notebooks/Ion/stack1/keras_fold3_best_loss.h5

Epoch 00002: val_accuracy improved from 0.95613 to 0.96522, saving model to ./drive/My Drive/Colab Notebooks/Ion/stack1/keras_fold3_best_score.h5
94/94 [==============================] - 52s 558ms/step - loss: 0.1144 - accuracy: 0.9627 - val

Training fold 3 completed. macro f1 score : 0.94020
[##### Running Fold: 3 #####] done in 5690 s
##### CV Score: 0.9410183626324503 #####
